In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_benchmark/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 'rnd_padding', 
    'aug_padding', "zoom_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/'
column = "EC number"
task = "task1/"

n_folds = 10

dicti = creating_dict()
max_lenn = 1000
n_class = 2 #number of classes to output
dict_size = len(dicti)
batch_size = 54
epochss = 200
callbacks_list = [None, None, None, None, False, False, False, False]
final_act = "softmax"
drop_per = 0.2 #Input dropout
drop_hid = 0.5
n_neur = [314,77, 8]

##### Each one of the following groups of parameters correspond to one of the architectures.
##### They must be uncommented according to the studied architecture in each case

############### only_denses architecture
architecture = "only_denses"
folder_arch = 'EC_number/archaea/3denses/bio_neurons/'
pool_size = None
n_filt = None
kernel_size = None
model = building_3dense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)

############## 1_conv architecture
#architecture = "conv_dense"
#folder_arch = 'EC_number/archaea/1conv/64filts_size5/'
#kernel_size = [5]
#n_filt = 64
#pool_size = None
#model = building_1convdense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, 
                                        #drop_hid, n_filt, kernel_size, 
                                        #final_act, folder)

############## stack_conv architecture
#architecture = "stack_conv"
#folder_arch = 'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#kernel_size = [1,3,5,9,15]
#n_filt = 10
#pool_size = 10
#model = building_stackconv_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
# kernel_size, pool_size, final_act, folder, optimizer)

model.summary()

In [3]:
lr = 0.0001
optimizer = Adam(lr=lr) 

### Loading stuff

In [4]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_split.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_split.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model  - Task 1

In [5]:
# Building generators

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task1', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 314)         8478      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 314)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 77)          24255     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 77)          0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000, 8)           624       
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder_arch, task, idx, callbacks_list, train_generator, val_generator,
                        architecture, max_lenn, dict_size, batch_size,
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val, 
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 314)         8478      
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 314)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 1000, 77)          24255     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 77)          0         
_________________________________________________________________
dense_7 (Dense)              (None, 1000, 8)           624       
__________

251/251 [==============================] - 6s 25ms/step - loss: 0.5974 - acc: 0.7041 - val_loss: 0.5765 - val_acc: 0.7324

Epoch 00021: val_acc improved from 0.73061 to 0.73235, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-021-0.7324.hdf5
Epoch 22/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5954 - acc: 0.7047 - val_loss: 0.5723 - val_acc: 0.7317

Epoch 00022: val_acc did not improve from 0.73235
Epoch 23/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5908 - acc: 0.7087 - val_loss: 0.5667 - val_acc: 0.7383

Epoch 00023: val_acc improved from 0.73235 to 0.73829, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-023-0.7383.hdf5
Epoch 24/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5897 - acc: 0.7081 - val_loss: 0.5630 - val_acc: 0.7372

Epo

251/251 [==============================] - 6s 25ms/step - loss: 0.5065 - acc: 0.7567 - val_loss: 0.4033 - val_acc: 0.8553

Epoch 00071: val_acc improved from 0.85360 to 0.85535, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-071-0.8553.hdf5
Epoch 72/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5056 - acc: 0.7606 - val_loss: 0.4028 - val_acc: 0.8564

Epoch 00072: val_acc improved from 0.85535 to 0.85639, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-072-0.8564.hdf5
Epoch 73/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5075 - acc: 0.7580 - val_loss: 0.4010 - val_acc: 0.8578

Epoch 00073: val_acc improved from 0.85639 to 0.85779, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improveme

251/251 [==============================] - 6s 25ms/step - loss: 0.4939 - acc: 0.7679 - val_loss: 0.3903 - val_acc: 0.8651

Epoch 00104: val_acc did not improve from 0.87142
Epoch 105/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4943 - acc: 0.7656 - val_loss: 0.3903 - val_acc: 0.8651

Epoch 00105: val_acc did not improve from 0.87142
Epoch 106/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4944 - acc: 0.7649 - val_loss: 0.3888 - val_acc: 0.8697

Epoch 00106: val_acc did not improve from 0.87142
Epoch 107/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4913 - acc: 0.7674 - val_loss: 0.3870 - val_acc: 0.8704

Epoch 00107: val_acc did not improve from 0.87142
Epoch 108/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4924 - acc: 0.7638 - val_loss: 0.3873 - val_acc: 0.8714

Epoch 00108: val_acc did not improve from 0.87142
Epoch 109/200
251/251 [==============================] - 6s 25ms/step - loss: 0


Epoch 00184: val_acc improved from 0.88085 to 0.88120, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-184-0.8812.hdf5
Epoch 185/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4681 - acc: 0.7802 - val_loss: 0.3643 - val_acc: 0.8816

Epoch 00185: val_acc improved from 0.88120 to 0.88155, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-185-0.8816.hdf5
Epoch 186/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4694 - acc: 0.7800 - val_loss: 0.3643 - val_acc: 0.8805

Epoch 00186: val_acc did not improve from 0.88155
Epoch 187/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4762 - acc: 0.7758 - val_loss: 0.3669 - val_acc: 0.8802

Epoch 00187: val_acc did not improve from 0.88155
Epoch 188/200
251/251 [==============================] - 6s 25ms/step - 

...

251/251 [==============================] - 7s 28ms/step - loss: 0.5306 - acc: 0.7413 - val_loss: 0.4482 - val_acc: 0.8267

Epoch 00150: val_acc did not improve from 0.82739
Epoch 151/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5229 - acc: 0.7446 - val_loss: 0.4461 - val_acc: 0.8288

Epoch 00151: val_acc improved from 0.82739 to 0.82879, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/5/weights-improvement-151-0.8288.hdf5
Epoch 152/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5271 - acc: 0.7463 - val_loss: 0.4461 - val_acc: 0.8288

Epoch 00152: val_acc did not improve from 0.82879
Epoch 153/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5228 - acc: 0.7478 - val_loss: 0.4461 - val_acc: 0.8288

Epoch 00153: val_acc did not improve from 0.82879
Epoch 154/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5282 - acc: 0.7394 - val_loss


Epoch 00190: val_acc did not improve from 0.83194
Epoch 191/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5187 - acc: 0.7483 - val_loss: 0.4434 - val_acc: 0.8270

Epoch 00191: val_acc did not improve from 0.83194
Epoch 192/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5164 - acc: 0.7477 - val_loss: 0.4397 - val_acc: 0.8305

Epoch 00192: val_acc did not improve from 0.83194
Epoch 193/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5170 - acc: 0.7507 - val_loss: 0.4390 - val_acc: 0.8312

Epoch 00193: val_acc did not improve from 0.83194
Epoch 194/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5224 - acc: 0.7432 - val_loss: 0.4438 - val_acc: 0.8267

Epoch 00194: val_acc did not improve from 0.83194
Epoch 195/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5168 - acc: 0.7455 - val_loss: 0.4437 - val_acc: 0.8274

Epoch 00195: val_acc did not improve from 0.83194
Epoch 196/200


251/251 [==============================] - 7s 27ms/step - loss: 0.6202 - acc: 0.6905 - val_loss: 0.6259 - val_acc: 0.7058

Epoch 00015: val_acc improved from 0.70405 to 0.70580, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-015-0.7058.hdf5
Epoch 16/200
251/251 [==============================] - 7s 28ms/step - loss: 0.6184 - acc: 0.6884 - val_loss: 0.6226 - val_acc: 0.7089

Epoch 00016: val_acc improved from 0.70580 to 0.70894, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-016-0.7089.hdf5
Epoch 17/200
251/251 [==============================] - 7s 27ms/step - loss: 0.6177 - acc: 0.6918 - val_loss: 0.6209 - val_acc: 0.7093

Epoch 00017: val_acc improved from 0.70894 to 0.70929, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-


Epoch 00040: val_acc improved from 0.74109 to 0.74284, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-040-0.7428.hdf5
Epoch 41/200
251/251 [==============================] - 7s 27ms/step - loss: 0.5911 - acc: 0.7031 - val_loss: 0.5586 - val_acc: 0.7446

Epoch 00041: val_acc improved from 0.74284 to 0.74458, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-041-0.7446.hdf5
Epoch 42/200
251/251 [==============================] - 7s 27ms/step - loss: 0.5905 - acc: 0.7040 - val_loss: 0.5555 - val_acc: 0.7467

Epoch 00042: val_acc improved from 0.74458 to 0.74668, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-042-0.7467.hdf5
Epoch 43/200
251/251 [==============================] - 7s 27ms/step - loss: 0.5869 - acc: 0.7052 - val_lo

251/251 [==============================] - 7s 28ms/step - loss: 0.5605 - acc: 0.7186 - val_loss: 0.5097 - val_acc: 0.7736

Epoch 00069: val_acc did not improve from 0.77463
Epoch 70/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5608 - acc: 0.7243 - val_loss: 0.5086 - val_acc: 0.7739

Epoch 00070: val_acc did not improve from 0.77463
Epoch 71/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5620 - acc: 0.7231 - val_loss: 0.5060 - val_acc: 0.7778

Epoch 00071: val_acc improved from 0.77463 to 0.77778, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/rnd_padding/6/weights-improvement-071-0.7778.hdf5
Epoch 72/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5562 - acc: 0.7230 - val_loss: 0.5051 - val_acc: 0.7771

Epoch 00072: val_acc did not improve from 0.77778
Epoch 73/200
251/251 [==============================] - 7s 28ms/step - loss: 0.5620 - acc: 0.7209 - val_loss: 0.